In [1]:
import pandas as pd
import tensorflow as tf
import classification_utils
import numpy as np

2022-11-21 23:37:10.263329: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-21 23:37:10.534967: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-21 23:37:10.535001: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-21 23:37:11.750849: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
df = pd.read_csv("./dataset/users_df_dataset_cleaned_with_indicators.csv")
# TODO check if it is the latest dataset

In [3]:
df.columns.values.tolist()

['user_id',
 'name',
 'lang',
 'bot',
 'created_at',
 'statuses_count',
 'account_age_in_days',
 'number_of_tweets',
 'account_average_tweets_per_day',
 'avg_tweets_per_actual_day',
 'day_with_most_tweets',
 'max_number_of_tweet_in_a_day',
 'entropy_for_day',
 'entropy_for_hour',
 'entropy_for_minute',
 'avg_hashtags',
 'avg_text_length',
 'avg_mentions',
 'avg_special_char_in_text',
 'total_likes',
 'avt_favorite_count',
 'total_replies',
 'avt_reply_count',
 'total_retweet_count',
 'account_discussion_creation_ratio',
 'tweet_num_likes_ratio',
 'tweet_num_replies_ratio']

In [4]:
to_drop = ["user_id", "name", "created_at"]
to_discretize = ["lang"]

df.drop(to_drop, axis=1, inplace=True)
df = classification_utils.discretize_data(df, to_discretize)
df.drop(to_discretize, axis=1, inplace=True)
# TODO after the dataset update check again

In [5]:
df.head()

,bot,statuses_count,account_age_in_days,number_of_tweets,account_average_tweets_per_day,avg_tweets_per_actual_day,day_with_most_tweets,max_number_of_tweet_in_a_day,entropy_for_day,entropy_for_hour,...,avg_special_char_in_text,total_likes,avt_favorite_count,total_replies,avt_reply_count,total_retweet_count,account_discussion_creation_ratio,tweet_num_likes_ratio,tweet_num_replies_ratio,lang_num
0,1,76,1314,126,0.057839,4.666667,2019-04-28,18,2.903992,2.762944,...,14.563492,5,0.039683,0,0.0,5,25.200000,25.200000,0.0,5
1,0,54,1311,116,0.041190,4.640000,2019-04-28,17,2.942683,2.881816,...,15.336207,6,0.051724,0,0.0,3,38.666667,19.333333,0.0,5
2,1,3,2709,4,0.001107,2.000000,2016-06-10,3,1.000000,0.918296,...,5.250000,0,0.000000,0,0.0,0,0.000000,0.000000,0.0,5
3,1,50,2080,1358,0.024038,8.035503,2019-11-01,143,3.543607,1.320785,...,19.241532,252,0.185567,0,0.0,1187,1.144061,5.388889,0.0,12
4,0,7085,1198,3434,5.914023,28.380165,2020-02-21,50,4.253096,1.705484,...,15.024461,213,0.062027,0,0.0,60,57.233333,16.122066,0.0,5


In [6]:
from sklearn.model_selection import train_test_split

label = df.pop("bot")

train_ratio = 0.70
validation_ratio = 0.20
test_ratio = 0.10

x_train, x_test, y_train, y_test = train_test_split(df, label, stratify =label, test_size= 1 - train_ratio)
x_val, x_test, y_val, y_test = train_test_split(x_train, y_train, test_size=test_ratio/(test_ratio + validation_ratio) )




In [7]:
x_train = np.asarray(x_train)
x_val = np.asarray(x_val)
x_test = np.asarray(x_test)
y_train = np.asarray(y_train)
y_val = np.asarray(y_val)
y_test = np.asarray(y_test)

In [8]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(1, len(df.columns))),
  tf.keras.layers.Dense(32, activation='sigmoid'),  
  tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adamax',
              loss='mean_squared_error',
              metrics=['accuracy'])

2022-11-21 23:37:13.925241: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-21 23:37:13.925283: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-21 23:37:13.925312: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mpc): /proc/driver/nvidia/version does not exist
2022-11-21 23:37:13.925896: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
y_train = np.reshape(y_train, (y_train.shape[0], 1, y_train.shape[1]))

history = model.fit(x_train, y_train,
                    epochs=80,
                    batch_size=512,
                    validation_split=0.2)

IndexError: tuple index out of range